# FinTech Assignment 2 : Business models and Application

#### Business Analytics & Management
#### Rotterdam School of Management, Erasmus University

Author: Yanbo Li

# Personalized Financial News and Advisory Platform (NAP) MVP

This notebook demonstrates a Minimum Viable Product (MVP) for the NAP system, which fetches financial news, summarizes articles, and performs sentiment analysis.

## Setup and Imports

First, let's import the necessary libraries and set up our API keys.

In [49]:
import os
import requests
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
import torch

# Set up API keys
NEWSAPI_KEY = "YOUR OWN API"
HUGGINGFACE_API_TOKEN = "YOUR OWN API"

# Set up Hugging Face API headers
headers = {"Authorization": f"Bearer {HUGGINGFACE_API_TOKEN}"}

# Initialize models and tokenizers
summarization_model = "facebook/bart-large-cnn"
sentiment_model = "distilbert-base-uncased-finetuned-sst-2-english"

summarization_tokenizer = AutoTokenizer.from_pretrained(summarization_model)
summarization_model = AutoModelForSeq2SeqLM.from_pretrained(summarization_model)

sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(sentiment_model)

## Fetching Financial News

We'll use the NewsAPI to fetch recent financial news articles.

In [58]:
import requests

def fetch_financial_news(query="finance", num_articles=5):
    url = f"https://newsapi.org/v2/everything?q={query}&language=en&sortBy=publishedAt&pageSize={num_articles}"
    headers = {"X-Api-Key": NEWSAPI_KEY}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()['articles']
    else:
        print(f"Error fetching news: {response.status_code}")
        print(f"Response content: {response.text}")
        return []

## Processing Articles

Now, let's create functions to summarize articles and analyze their sentiment using Hugging Face's API.

In [89]:
def summarize_article(text):
    API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"
    payload = {"inputs": text, "parameters": {"max_length": 100, "min_length": 30}}
    response = requests.post(API_URL, headers=headers, json=payload)
    print(f"Summarization Status Code: {response.status_code}")
    if response.status_code == 200:
        result = response.json()
        if isinstance(result, list) and len(result) > 0 and 'summary_text' in result[0]:
            return result[0]['summary_text']
    print(f"Summarization failed. Input text: {text[:100]}...")  # Print first 100 characters of input
    print(f"Response content: {response.text}")
    return "Summary not available"

def analyze_sentiment(text):
    API_URL = "https://api-inference.huggingface.co/models/distilbert-base-uncased-finetuned-sst-2-english"
    payload = {"inputs": text[:512]}
    response = requests.post(API_URL, headers=headers, json=payload)
    if response.status_code == 200:
        result = response.json()
        if isinstance(result, list) and len(result) > 0 and isinstance(result[0], list) and len(result[0]) > 0:
            sentiment = result[0][0]
            return sentiment['label'], sentiment['score']
    print(f"Error in sentiment analysis: {response.status_code}")
    print(f"Response content: {response.text}")
    return "UNKNOWN", 0.0

def process_article(article):
    title = article.get('title', 'No title')
    content = article.get('content') or article.get('description', 'No content')
    
    print(f"Processing article: {title}")
    print(f"Content to summarize: {content[:100]}...")  # Print first 100 characters
    
    summary = summarize_article(content)
    sentiment_label, sentiment_score = analyze_sentiment(content)
    
    print(f"Generated summary: {summary[:100]}...")  # Print first 100 characters of summary
    
    return {
        'title': title,
        'summary': summary,
        'sentiment': sentiment_label,
        'sentiment_score': sentiment_score,
        'url': article.get('url', 'No URL')
    }

## Main NAP Function

Let's create our main function to run the NAP MVP.

In [100]:
def run_nap(query="finance", num_articles=5):
    articles = fetch_financial_news(query, num_articles)
    
    if not articles:
        print("No articles fetched. Returning empty DataFrame.")
        return pd.DataFrame()

    processed_articles = []
    for article in articles:
        try:
            processed = process_article(article)
            processed_articles.append(processed)
        except Exception as e:
            print(f"Error processing article: {str(e)}")
    
    df = pd.DataFrame(processed_articles)
    return df

## Running the MVP

Now, let's run our NAP MVP and display the results.

In [102]:
# Run NAP
results = run_nap(query="finance", num_articles=5)

# Display results
print(results)

# Optionally, save to CSV
results.to_csv("nap_results.csv", index=False)
print("Results saved to nap_results.csv")

Processing article: Kenyan Youth Embrace Bitcoin Amid Deadly Protests Over Finance Bill
Content to summarize: Kenyan Bitcoiner protesting the controversial Finance Bill 2024
Felix Mukungu
On Tuesday, June 25,...
Summarization Status Code: 200
Generated summary: On Tuesday, June 25, thousands took to the streets of Nairobi, the capital of Kenya, in a youth-led ...
Processing article: CPS Announces $436.31 Million Senior Subordinate Asset-Backed Securitization
Content to summarize: LAS VEGAS, Nevada, June 26, 2024 (GLOBE NEWSWIRE) -- Consumer Portfolio Services, Inc. (Nasdaq: CPSS...
Summarization Status Code: 200
Generated summary: Consumer Portfolio Services, Inc. (Nasdaq: CPSS) (CPS or the Company) announced the closing of its t...
Processing article: Lula Cements Brazil Central Bank’s Switch to Continuous CPI Target
Content to summarize: (Bloomberg) -- Brazil is changing its inflation targeting regime and setting a 3% goal for continuou...
Summarization Status Code: 200
Generated sum

# Creating an Interactive Dashboard for NAP Results

In this section, we'll create an interactive dashboard to visualize and explore the results from our Personalized Financial News and Advisory Platform (NAP).


In [104]:
from IPython.display import HTML, display
import json
import plotly.graph_objs as go
import random
import yfinance as yf
from datetime import datetime, timedelta

def get_stock_data(ticker="^IXIC", period="1mo"):
    stock = yf.Ticker(ticker)
    hist = stock.history(period=period)
    return hist

def create_stock_chart(data, title):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Close Price'))
    fig.update_layout(
        title=title,
        xaxis_title='Date',
        yaxis_title='Price',
        height=400,
        margin=dict(l=0, r=0, t=40, b=0)
    )
    return fig

def save_dashboard_html(results, filename='dashboard.html'):
    # Create a simple bar chart for sentiment scores
    fig_sentiment = go.Figure(data=[go.Bar(x=results['title'].tolist(), y=results['sentiment_score'].tolist(), marker_color='#4CAF50')])
    fig_sentiment.update_layout(title='Article Sentiment Scores', height=300, margin=dict(l=0, r=0, t=40, b=0))

    # Convert the Plotly figures to JSON
    plot_json_sentiment = fig_sentiment.to_json()

    # Generate random "read time" for each article (3-10 minutes)
    read_times = [random.randint(3, 10) for _ in range(len(results))]

    html_content = """
    <html>
    <head>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.6.0/jquery.min.js"></script>
        <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@300;400;700&display=swap" rel="stylesheet">
        <style>
            body {
                font-family: 'Roboto', sans-serif;
                margin: 0;
                padding: 0;
                display: flex;
                background-color: #f0f2f5;
            }
            #sidebar {
                width: 300px;
                height: 100vh;
                overflow-y: auto;
                background-color: #ffffff;
                box-shadow: 2px 0 5px rgba(0,0,0,0.1);
                padding: 20px;
            }
            #main-content {
                flex-grow: 1;
                padding: 20px;
                height: 100vh;
                overflow-y: auto;
            }
            .article-item {
                cursor: pointer;
                padding: 15px;
                margin-bottom: 10px;
                background-color: #ffffff;
                border-radius: 8px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                transition: all 0.3s ease;
            }
            .article-item:hover {
                transform: translateY(-3px);
                box-shadow: 0 4px 6px rgba(0,0,0,0.1);
            }
            .article-item.active {
                background-color: #e7f3ff;
                border-left: 4px solid #1877f2;
            }
            .article-title {
                font-weight: bold;
                margin-bottom: 5px;
            }
            .article-metadata {
                font-size: 0.8em;
                color: #65676b;
            }
            #article-details {
                background-color: #ffffff;
                padding: 20px;
                border-radius: 8px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                margin-bottom: 20px;
            }
            .sentiment-pill {
                display: inline-block;
                padding: 5px 10px;
                border-radius: 20px;
                font-size: 0.8em;
                font-weight: bold;
                margin-right: 10px;
            }
            .sentiment-POSITIVE { background-color: #e6f3e6; color: #2e7d32; }
            .sentiment-NEGATIVE { background-color: #fde7e7; color: #c62828; }
            .sentiment-NEUTRAL { background-color: #e8eaf6; color: #3f51b5; }
            .header {
                background-color: #1877f2;
                color: white;
                padding: 20px;
                font-size: 24px;
                font-weight: bold;
                margin-bottom: 20px;
                border-radius: 8px;
            }
            .chart-container {
                background-color: #ffffff;
                padding: 20px;
                border-radius: 8px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                margin-bottom: 20px;
            }
            .chart-controls {
                display: flex;
                justify-content: space-between;
                margin-bottom: 10px;
            }
            select {
                padding: 5px;
                border-radius: 5px;
                border: 1px solid #ccc;
            }
        </style>
    </head>
    <body>
        <div id="sidebar">
            <div class="header">NAP News Feed</div>
            <div id="article-list"></div>
        </div>
        <div id="main-content">
            <div class="chart-container">
                <div class="chart-controls">
                    <select id="index-select">
                        <option value="^IXIC">NASDAQ Composite</option>
                        <option value="^DJI">Dow Jones Industrial Average</option>
                        <option value="^GSPC">S&P 500</option>
                        <option value="^RUT">Russell 2000</option>
                        <option value="^NYA">NYSE Composite</option>
                    </select>
                    <select id="timeframe-select">
                        <option value="1wk">1 Week</option>
                        <option value="1mo" selected>1 Month</option>
                        <option value="3mo">3 Months</option>
                        <option value="6mo">6 Months</option>
                        <option value="1y">1 Year</option>
                    </select>
                </div>
                <div id="stock_chart"></div>
            </div>
            <div id="article-details"></div>
            <div class="chart-container">
                <div id="sentiment_chart"></div>
            </div>
        </div>
        <script>
            var results = """ + results.to_json(orient='records') + """;
            var readTimes = """ + json.dumps(read_times) + """;
            
            function updateDisplay(index) {
                $('.article-item').removeClass('active');
                $('#article-' + index).addClass('active');
                var row = results[index];
                var sentimentClass = 'sentiment-' + row.sentiment;
                var details = `
                    <h2>${row.title}</h2>
                    <p class="article-metadata">
                        <span class="sentiment-pill ${sentimentClass}">${row.sentiment}</span>
                        <span>${readTimes[index]} min read</span>
                    </p>
                    <p><b>Summary:</b> ${row.summary}</p>
                    <p><b>Sentiment Score:</b> ${row.sentiment_score.toFixed(2)}</p>
                    <p><a href="${row.url}" target="_blank">Read full article</a></p>
                `;
                $('#article-details').html(details);
            }
            
            var articleList = $('#article-list');
            results.forEach((article, index) => {
                var sentimentClass = 'sentiment-' + article.sentiment;
                articleList.append(`
                    <div class="article-item" id="article-${index}">
                        <div class="article-title">${article.title}</div>
                        <div class="article-metadata">
                            <span class="sentiment-pill ${sentimentClass}">${article.sentiment}</span>
                            <span>${readTimes[index]} min read</span>
                        </div>
                    </div>
                `);
            });
            
            $('.article-item').click(function() {
                var index = $(this).attr('id').split('-')[1];
                updateDisplay(index);
            });
            
            updateDisplay(0);  // Initialize with the first article

            var plotlyDataSentiment = """ + plot_json_sentiment + """;
            Plotly.newPlot('sentiment_chart', plotlyDataSentiment.data, plotlyDataSentiment.layout);

            function updateStockChart() {
                var index = $('#index-select').val();
                var timeframe = $('#timeframe-select').val();
                var indexName = $('#index-select option:selected').text();
                
                fetch(`https://query1.finance.yahoo.com/v8/finance/chart/${index}?range=${timeframe}&interval=1d`)
                    .then(response => response.json())
                    .then(data => {
                        var prices = data.chart.result[0].indicators.quote[0].close;
                        var timestamps = data.chart.result[0].timestamp.map(t => new Date(t * 1000));
                        
                        var trace = {
                            x: timestamps,
                            y: prices,
                            type: 'scatter',
                            mode: 'lines',
                            name: 'Close Price'
                        };
                        
                        var layout = {
                            title: `${indexName} - Last ${$('#timeframe-select option:selected').text()}`,
                            xaxis: { title: 'Date' },
                            yaxis: { title: 'Price' },
                            height: 400,
                            margin: { l: 50, r: 50, t: 40, b: 50 }
                        };
                        
                        Plotly.newPlot('stock_chart', [trace], layout);
                    });
            }

            $('#index-select, #timeframe-select').change(updateStockChart);
            
            // Initial stock chart update
            updateStockChart();
        </script>
    </body>
    </html>
    """
    
    with open(filename, 'w') as f:
        f.write(html_content)
    
    print(f"Dashboard saved as {filename}")
    return HTML(f'<a href="{filename}" target="_blank">Open Dashboard</a>')



In [105]:
# Save and display link to the dashboard
display(save_dashboard_html(results))

Dashboard saved as dashboard.html
